In [2]:
# ==========================================
# 🌍 FINAL CLEAN AQI MODEL TRAINING
# ==========================================

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# ------------------------------------------
# 1️⃣ Load Dataset
# ------------------------------------------
df = pd.read_csv("data.csv" , encoding = 'latin1')
df = df.sample(150000, random_state = 42).reset_index(drop=True) 
# ------------------------------------------
# 2️⃣ Drop Unnecessary + High Cardinality Columns
# ------------------------------------------
columns_to_drop = [
    'pm2_5',
    'spm',
    'agency',
    'stn_code',
    'date',
    'sampling_date',
    'location',                     # DROP THIS
    'location_monitoring_station'   # DROP THIS
]

df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])


# ------------------------------------------
# 4️⃣ Handle Missing Values
# ------------------------------------------
numeric_cols = ['so2', 'no2', 'rspm']

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Fill categorical
if 'state' in df.columns:
    df['state'] = df['state'].fillna("Unknown")

if 'type' in df.columns:
    df['type'] = df['type'].fillna("Unknown")

# ------------------------------------------
# 5️⃣ Define Features & Target
# ------------------------------------------
X = df.drop(columns=['rspm'])
y = df['rspm']

# ------------------------------------------
# 6️⃣ Encode Only Low-Cardinality Columns
# ------------------------------------------
X = pd.get_dummies(
    X,
    columns=[col for col in ['state', 'type'] if col in X.columns],
    drop_first=True
)

# Save feature list
feature_columns = X.columns.tolist()

# ------------------------------------------
# 7️⃣ Train-Test Split
# ------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# ------------------------------------------
# 8️⃣ Train Models
# ------------------------------------------
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(
        n_estimators=50,
        max_depth=12,
        n_jobs=-1,
        random_state=42
    ),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42)
}

best_model = None
best_score = -1

print("\n📊 Model Performance Results:")

for name, model in models.items():

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    r2 = r2_score(y_test, predictions)
    rmse = np.sqrt(mean_squared_error(y_test, predictions))

    print(f"\nModel: {name}")
    print("R2 Score:", round(r2, 4))
    print("RMSE:", round(rmse, 4))

    if r2 > best_score:
        best_score = r2
        best_model = model
        best_model_name = name

# ------------------------------------------
# 9️⃣ Final Best Model
# ------------------------------------------
print("\n🏆 Best Model Selected:", best_model_name)
print("Best R2 Score:", round(best_score, 4))

if best_score >= 0.8:
    print("Model Performance: Excellent 🔥")
elif best_score >= 0.6:
    print("Model Performance: Good 👍")
elif best_score >= 0.4:
    print("Model Performance: Moderate ⚠️")
else:
    print("Model Performance: Needs Improvement ❌")

# ------------------------------------------
# 🔟 Save Model
# ------------------------------------------
joblib.dump(best_model, "aqi_model.pkl")
joblib.dump(feature_columns, "model_features.pkl")

print("\n💾 Model and feature list saved successfully!")

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14220\442578689.py:17: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv" , encoding = 'latin1')



📊 Model Performance Results:

Model: Linear Regression
R2 Score: 0.315
RMSE: 59.4456

Model: Random Forest
R2 Score: 0.3999
RMSE: 55.6426

Model: Gradient Boosting
R2 Score: 0.36
RMSE: 57.4591

🏆 Best Model Selected: Random Forest
Best R2 Score: 0.3999
Model Performance: Needs Improvement ❌

💾 Model and feature list saved successfully!
